In [1]:
import os
import sys

notebook_path = os.getcwd()
project_path = os.path.abspath(os.path.join(notebook_path, "..", "..", ".."))
print(f"Project path: {project_path}")
# Add the project path to the system path
sys.path.insert(0, project_path)

Project path: c:\Users\dalej\Documents\_Coding\DragonRegen


In [2]:
from src.MetaFort.SysLogs.DatabaseEngine import DatabaseEngine
from src.DataCreator.SchemaGenerators.SchemaMSSQL import SchemaMSSQL

# create database engine object and connection
driver = "ODBC Driver 17 for SQL Server"
server = 'localhost\\SQLEXPRESS' 
# server = "Andrew=PC\\SQLEXPRESS"
database = "MetaFort"
conn_str = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)
db_engine = DatabaseEngine(conn_str)
db_engine.connect()
print("Connected to the database.")

Connected to the database.


In [3]:
# Initialize logging table for dynamic sql executions
s_sql_col = SchemaMSSQL.generate_schema_sql(DatabaseEngine.d_log_tables)
print(s_sql_col)

{'dyn_sql_execution_log': '    [execution_id] INT NOT NULL IDENTITY(1,1),\n    [sql_command] NVARCHAR(MAX) NOT NULL,\n    [sql_params] NVARCHAR(MAX) NOT NULL,\n    [start_time] NVARCHAR(MAX) NOT NULL,\n    [stop_time] NVARCHAR(MAX) NOT NULL,\n    [status] NVARCHAR(MAX) NOT NULL,\n    [user_name] NVARCHAR(MAX) NOT NULL,\n    [process_id] NVARCHAR(MAX) NOT NULL,\n    [process_login_time] DATETIME2 NOT NULL,\n    [error_message] NVARCHAR(MAX) NULL,\n    [error_timestamp] NVARCHAR(MAX) NULL,\n    [metadata] NVARCHAR(MAX) NULL'}


In [4]:
# execute with logging using the database engine
db_engine.create_table('dyn_sql_execution_log', s_sql_col['dyn_sql_execution_log'])
print("Created table for dynamic sql executions")

Created table for dynamic sql executions
